In [ ]:
import pandas as pd
import numpy as np
import wget
from tqdm import tqdm
import cv2 
import matplotlib.pyplot as plt
from PIL import Image
import glob 

import tensorflow as tf

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Segmenting Plant Images

In [ ]:
img = cv2.imread("milkweed/images/4170.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

low_green = np.array([30, 0, 0])
high_green = np.array([95, 255, 255])

mask = cv2.inRange(hsv, low_green, high_green)

kernel = np.ones((5, 5), np.uint8)
mask = cv2.erode(mask, kernel) 
mask = cv2.erode(mask, kernel) 
mask = cv2.dilate(mask, kernel) 

res = cv2.bitwise_and(img,img, mask= mask)

nb_blobs, im_with_separated_blobs, stats, _ = cv2.connectedComponentsWithStats(mask)
sizes = stats[:, -1]
sizes = sizes[1:]
nb_blobs -= 1

min_size = 20500  

im_result = np.zeros_like(im_with_separated_blobs)
for blob in range(nb_blobs):
    if sizes[blob] >= min_size:
        im_result[im_with_separated_blobs == blob + 1] = 255

res = cv2.bitwise_and(img,img, mask= np.array(im_result).astype(np.uint8))

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (10,10))
fig.tight_layout()
ax1.imshow(img)
ax2.imshow(255 - mask, cmap="Greys")
ax3.imshow(res)

## Greenness Identification

In [ ]:
img = cv2.imread("milkweed/images/4198.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img / 255

ExG = 2*img[:, :, 1] - img[:, :, 0] - img[:, :, 2]
ExGR = ExG - (1.4*img[:, :, 0] - img[:, :, 1])
VEG = img[:, :, 1] / ((img[:, :, 0]**0.667)*(img[:, :, 2]**0.333))
CIVE = (0.441*img[:, :, 0]) - (0.881*img[:, :, 1]) + (0.385*img[:, :, 2]) + 18.78745
COM = (0.25*ExG) + (0.30*ExGR) + (0.33*CIVE) + (0.12*VEG)

mask = np.where(COM < 7, 1, 0)

res = cv2.bitwise_and(img,img, mask= np.array(mask).astype(np.uint8))

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (10,10))
fig.tight_layout()
ax1.imshow(img)
ax2.imshow(255 - mask, cmap="Greys")
ax3.imshow(res)